Written by Evan Feinberg and Bharath Ramsundar

Copyright 2016, Stanford University

#Welcome to the deepchem tutorial. In this iPython Notebook, one can follow along with the code below to learn how to fit machine learning models with rich predictive power on chemical datasets.  

Overview:

In this tutorial, you will trace an arc from loading a raw dataset to fitting a cutting edge ML technique for predicting binding affinities. This will be accomplished by writing simple commands to access the deepchem Python API, encompassing the following broad steps:

1. Loading a chemical dataset, consisting of a series of protein-ligand complexes.
2. Featurizing each protein-ligand complexes with various featurization schemes. 
3. Fitting a series of models with these featurized protein-ligand complexes.
4. Visualizing the results.

First, let's point to a "dataset" file. This can come in the format of a CSV file or Pandas DataFrame. Regardless
of file format, it must be columnar data, where each row is a molecular system, and each column represents
a different piece of information about that system. For instance, in this example, every row reflects a 
protein-ligand complex, and the following columns are present: a unique complex identifier; the SMILES string
of the ligand; the binding affinity (Ki) of the ligand to the protein in the complex; a Python `list` of all lines
in a PDB file for the protein alone; and a Python `list` of all lines in a ligand file for the ligand alone.

This should become clearer with the example.


In [2]:
dataset_file= "../datasets/pdbbind_core_df.pkl.gz"
from deepchem.utils.save import load_from_disk
dataset = load_from_disk(dataset_file)

Let's see what `dataset` looks like:

In [3]:
print("Type of dataset is: %s" % str(type(dataset)))
print(dataset[:5])
print("Shape of dataset is: %s" % str(dataset.shape))

Type of dataset is: <class 'pandas.core.frame.DataFrame'>
  pdb_id                                             smiles  \
0   2d3u        CC1CCCCC1S(O)(O)NC1CC(C2CCC(CN)CC2)SC1C(O)O   
1   3cyx  CC(C)(C)NC(O)C1CC2CCCCC2C[NH+]1CC(O)C(CC1CCCCC...   
2   3uo4        OC(O)C1CCC(NC2NCCC(NC3CCCCC3C3CCCCC3)N2)CC1   
3   1p1q                         CC1ONC(O)C1CC([NH3+])C(O)O   
4   3ag9  NC(O)C(CCC[NH2+]C([NH3+])[NH3+])NC(O)C(CCC[NH2...   

                                          complex_id  \
0    2d3uCC1CCCCC1S(O)(O)NC1CC(C2CCC(CN)CC2)SC1C(O)O   
1  3cyxCC(C)(C)NC(O)C1CC2CCCCC2C[NH+]1CC(O)C(CC1C...   
2    3uo4OC(O)C1CCC(NC2NCCC(NC3CCCCC3C3CCCCC3)N2)CC1   
3                     1p1qCC1ONC(O)C1CC([NH3+])C(O)O   
4  3ag9NC(O)C(CCC[NH2+]C([NH3+])[NH3+])NC(O)C(CCC...   

                                         protein_pdb  \
0  [HEADER    2D3U PROTEIN\n, COMPND    2D3U PROT...   
1  [HEADER    3CYX PROTEIN\n, COMPND    3CYX PROT...   
2  [HEADER    3UO4 PROTEIN\n, COMPND    3UO4 PROT...   
3 

Now that we're oriented, let's use ML to do some chemistry. 

So, step (2) will entail featurizing the dataset.

The available featurizations that come standard with deepchem are ECFP4 fingerprints, RDKit descriptors, NNScore-style bdescriptors, and hybrid binding pocket descriptors. Details can be found on ```deepchem.io```.

In [4]:
from deepchem.featurizers.fingerprints import CircularFingerprint
from deepchem.featurizers.basic import RDKitDescriptors
from deepchem.featurizers.nnscore import NNScoreComplexFeaturizer

compound_featurizers = [CircularFingerprint(size=1024)]
complex_featurizers = []

Note how we separate our featurizers into those that featurize individual chemical compounds, compound_featurizers, and those that featurize molecular complexes, complex_featurizers.

Now, let's perform the actual featurization. Calling ```featurizer.featurize()``` will return an instance of class ```FeaturizedSamples```. Internally, ```featurizer.featurize()``` (a) computes the user-specified features on the data, (b) transforms the inputs into X and y NumPy arrays suitable for ML algorithms, and (c) constructs a ```FeaturizedSamples()``` instance that has useful methods, such as an iterator, over the featurized data.

In [5]:
#Make a directory in which to store the featurized complexes.
import tempfile, shutil
feature_dir = tempfile.mkdtemp()
samples_dir = tempfile.mkdtemp()

In [6]:
from deepchem.featurizers.featurize import DataFeaturizer

In [7]:
featurizers = compound_featurizers + complex_featurizers
featurizer = DataFeaturizer(tasks=["label"],
                            smiles_field="smiles",
                            protein_pdb_field="protein_pdb",
                            ligand_pdb_field="ligand_pdb",
                            compound_featurizers=compound_featurizers,
                            complex_featurizers=complex_featurizers,
                            id_field="complex_id",
                            verbose=False)
featurized_samples = featurizer.featurize(dataset_file, feature_dir, samples_dir,
                                          shard_size=100)

Now, we conduct a train-test split. If you'd like, you can choose `splittype="scaffold"` instead to perform a train-test split based on Bemis-Murcko scaffolds.

In [8]:
splittype = "random"
train_dir, test_dir = tempfile.mkdtemp(), tempfile.mkdtemp()

train_samples, test_samples = featurized_samples.train_test_split(
    splittype, train_dir, test_dir)

We generate separate instances of the Dataset() object to hermetically seal the train dataset from the test dataset. This style lends itself easily to validation-set type hyperparameter searches, which we will illustate in a separate tutorial. 

In [9]:
from deepchem.utils.dataset import Dataset

In [10]:
train_dataset = Dataset(data_dir=train_dir, samples=train_samples, 
                        featurizers=featurizers, tasks=["label"])
test_dataset = Dataset(data_dir=test_dir, samples=test_samples, 
                       featurizers=featurizers, tasks=["label"])

ValueError: total size of new array must be unchanged

The performance of many ML algorithms hinges greatly on careful data preprocessing. Deepchem comes standard with a few options for such preprocessing.

In [ ]:
input_transforms = ["normalize", "truncate"]
output_transforms = ["normalize"]
train_dataset.transform(input_transforms, output_transforms)
test_dataset.transform(input_transforms, output_transforms)

Now, we're ready to do some learning! To set up a model, we will need: (a) a dictionary ```task_types``` that maps a task, in this case ```label```, i.e. the Ki, to the type of the task, in this case ```regression```. For the multitask use case, one will have a series of keys, each of which is a different task (Ki, solubility, renal half-life, etc.) that maps to a different task type (regression or classification).

To fit a deepchem model, first we instantiate one of the provided (or user-written) model classes. In this case, we have a created a convenience class to wrap around any ML model available in Sci-Kit Learn that can in turn be used to interoperate with deepchem. To instantiate an ```SklearnModel```, you will need (a) task_types, (b) model_params, another ```dict``` as illustrated below, and (c) a ```model_instance``` defining the type of model you would like to fit, in this case a ```RandomForestRegressor```.

In [11]:
from sklearn.ensemble import RandomForestRegressor
from deepchem.models.standard import SklearnModel

In [13]:
task_types = {"label": "regression"}
model_params = {"data_shape": train_dataset.get_data_shape()}

model = SklearnModel(task_types, model_params, model_instance=RandomForestRegressor())
model.fit(train_dataset)
model_dir = tempfile.mkdtemp()
model.save(model_dir)

[-0.11470787  0.76656954 -0.53229065 ..., -0.08084521 -0.11470787
  2.93683503]
(154, 1024)


In [14]:
from deepchem.utils.evaluate import Evaluator
import pandas as pd

In [15]:
evaluator = Evaluator(model, train_dataset, verbose=True)
with tempfile.NamedTemporaryFile() as train_csv_out:
  with tempfile.NamedTemporaryFile() as train_stats_out:
    _, train_r2score = evaluator.compute_model_performance(
        train_csv_out, train_stats_out)

evaluator = Evaluator(model, test_dataset, verbose=True)
with tempfile.NamedTemporaryFile() as test_csv_out:
  with tempfile.NamedTemporaryFile() as test_stats_out:
    _, test_r2score = evaluator.compute_model_performance(
        test_csv_out, test_stats_out)

train_test_performance = pd.concat([train_r2score, test_r2score])
train_test_performance["split"] = ["train", "test"]
train_test_performance

Saving predictions to <open file '<fdopen>', mode 'w+b' at 0x7fd5759de270>
Saving model performance scores to <open file '<fdopen>', mode 'w+b' at 0x7fd5759de1e0>
Saving predictions to <open file '<fdopen>', mode 'w+b' at 0x7fd5759de150>
Saving model performance scores to <open file '<fdopen>', mode 'w+b' at 0x7fd5759de0c0>


,task_name,r2_score,rms_error,split
0,label,0.779266,1.062205,train
0,label,0.201858,1.946661,test


# Hyperparameter Searches to Construct the Best Model.

This simple example, in few yet intuitive lines of code, traces the machine learning arc from featurizing a raw dataset to fitting and evaluating a model. 

In the next section, we illustrate ```deepchem```'s modularity, and thereby the ease with which one can explore different featurization schemes, different models, and combinations thereof, to achieve the best performance on a given dataset. 

In [18]:
class fingerprints32(CircularFingerprint):
    def __init__(self):
        super(fingerprints32, self).__init__(size=32)
        
class fingerprints64(CircularFingerprint):
    def __init__(self):
        super(fingerprints64, self).__init__(size=64)
        
class fingerprints128(CircularFingerprint):
    def __init__(self):
        super(fingerprints128, self).__init__(size=128)

class fingerprints256(CircularFingerprint):
    def __init__(self):
        super(fingerprints256, self).__init__(size=256)

class fingerprints512(CircularFingerprint):
    def __init__(self):
        super(fingerprints512, self).__init__(size=512)
        

In [ ]:
from imp import reload
import deepchem.featurizers.grid_featurizer
reload(deepchem.featurizers.grid_featurizer)
from deepchem.featurizers.grid_featurizer import GridFeaturizer 
compound_featurizers = [fingerprints32(), fingerprints64(), fingerprints128(), fingerprints256(), fingerprints512()]
complex_featurizers = [GridFeaturizer(voxel_width=16.0, feature_types="voxel_combined", voxel_feature_types=["ecfp",
                                      "splif", "hbond", "pi_stack", "cation_pi", "salt_bridge"], ecfp_power=5, splif_power=5,
                                     parallel=True, flatten=True)]

featurizers = compound_featurizers + complex_featurizers
featurizer = DataFeaturizer(tasks=["label"],
                            smiles_field="smiles",
                            protein_pdb_field="protein_pdb",
                            ligand_pdb_field="ligand_pdb",
                            compound_featurizers=compound_featurizers,
                            complex_featurizers=complex_featurizers,
                            id_field="complex_id",
                            verbose=False)
featurized_samples = featurizer.featurize(dataset_file, feature_dir, samples_dir,
                                          shard_size=100)
print("Done featurizing.")
train_dir, test_dir = tempfile.mkdtemp(), tempfile.mkdtemp()
splittype="scaffold"
train_samples, test_samples = featurized_samples.train_test_split(
    splittype, train_dir, test_dir)

task_types = {"label": "regression"}


performance = pd.DataFrame()
all_featurizers = compound_featurizers + complex_featurizers


In [26]:
dataset_file= "../datasets/pdbbind_core_df.pkl.gz"
from deepchem.utils.save import load_from_disk
from deepchem.featurizers.featurize import DataFeaturizer
dataset = load_from_disk(dataset_file)

from imp import reload
import deepchem.featurizers.grid_featurizer
reload(deepchem.featurizers.grid_featurizer)
from deepchem.featurizers.grid_featurizer import GridFeaturizer 
import deepchem.featurizers.featurize
reload(deepchem.featurizers.featurize)
from deepchem.featurizers.featurize import DataFeaturizer

compound_featurizers = []
complex_featurizers = [GridFeaturizer(voxel_width=16.0, feature_types="voxel_combined", voxel_feature_types=["ecfp",
                                      "splif", "hbond", "pi_stack", "cation_pi", "salt_bridge"], ecfp_power=5, splif_power=5,
                                     parallel=True, flatten=True)]

import tempfile
feature_dir = tempfile.mkdtemp()
samples_dir = tempfile.mkdtemp()

featurizers = compound_featurizers + complex_featurizers
featurizer = DataFeaturizer(tasks=["label"],
                            smiles_field="smiles",
                            protein_pdb_field="protein_pdb",
                            ligand_pdb_field="ligand_pdb",
                            compound_featurizers=compound_featurizers,
                            complex_featurizers=complex_featurizers,
                            id_field="complex_id",
                            verbose=False)
featurized_samples = featurizer.featurize(dataset_file, feature_dir, samples_dir,
                                          shard_size=100)

In [40]:
train_dir, test_dir = tempfile.mkdtemp(), tempfile.mkdtemp()
splittype="random"
train_samples, test_samples = featurized_samples.train_test_split(
    splittype, train_dir, test_dir)

task_types = {"label": "regression"}

performance = pd.DataFrame()
import deepchem.models.standard
reload(deepchem.models.standard)
from deepchem.models.standard import SklearnModel

for feature_type in complex_featurizers:
    train_dataset = Dataset(data_dir=train_dir, samples=train_samples, 
                        featurizers=[feature_type], tasks=["label"])
    test_dataset = Dataset(data_dir=test_dir, samples=test_samples, 
                       featurizers=[feature_type], tasks=["label"])


    input_transforms = ["normalize", "truncate"]
    output_transforms = ["normalize"]
    train_dataset.transform(input_transforms, output_transforms)
    test_dataset.transform(input_transforms, output_transforms)

    model_params = {"data_shape": train_dataset.get_data_shape()}

    model = SklearnModel(task_types, model_params, model_instance=RandomForestRegressor())
    model.fit(train_dataset)
    model_dir = tempfile.mkdtemp()
    model.save(model_dir)


    evaluator = Evaluator(model, train_dataset, verbose=True)
    with tempfile.NamedTemporaryFile() as train_csv_out:
      with tempfile.NamedTemporaryFile() as train_stats_out:
        _, train_r2score = evaluator.compute_model_performance(
            train_csv_out, train_stats_out)

    evaluator = Evaluator(model, test_dataset, verbose=True)
    with tempfile.NamedTemporaryFile() as test_csv_out:
      with tempfile.NamedTemporaryFile() as test_stats_out:
        _, test_r2score = evaluator.compute_model_performance(
            test_csv_out, test_stats_out)

    train_test_performance = pd.concat([train_r2score, test_r2score])
    train_test_performance["split"] = ["train", "test"]
    train_test_performance["featurizer"] = [feature_type.__class__, feature_type.__class__]
    performance = pd.concat([performance, train_test_performance])
performance

[ 0.32431096 -1.02340963  0.57996428  0.27124449  0.16904514  2.13678197
 -1.08830859 -0.7418619  -0.10299668 -1.08157923 -0.5304159   0.63179685
 -0.43617623  1.28354512  1.42701175 -0.64064843  0.17870311 -1.08909977
  0.47461001  0.01907988 -0.94183639 -0.41402442 -1.03543363 -0.65718741
  0.43465308 -1.33269838  3.02004016  0.1348471  -0.79868815 -0.39198252
 -1.07535467  0.8779346  -0.29397237 -0.31976474 -0.39872799 -0.21821789
 -0.15839699 -0.44167439 -0.340807   -0.37711031  2.96871564 -0.38201798
 -0.30663372 -0.41067527 -0.36139007 -0.29070095 -0.33471631  1.88839555
 -0.31898089 -0.26325717 -0.23408229 -0.23929002 -0.34836651 -0.17490779
 -0.1409523  -0.24913644 -0.2773501  -0.43037564 -0.4020422  -0.23918244
 -0.39629892 -0.39735971 -0.27140133 -0.26752678  1.4076989   0.74043574
  0.29456017  1.17029526 -0.36757229 -0.91705478 -0.04253831 -0.22940977
  1.88584006 -0.05218315  0.52334107 -0.66417771 -1.09761972 -0.46917676
 -0.8662737   0.18682163  0.93333363 -0.243436   -0

,task_name,r2_score,rms_error,split,featurizer
0,label,0.871184,0.803785,train,<class 'deepchem.featurizers.grid_featurizer.G...
0,label,0.236349,1.949254,test,<class 'deepchem.featurizers.grid_featurizer.G...


While we have computed three separate featurizations -- ECFP4, RDKitDescriptors, and NNScore -- in this example we choose to use the RDKitDescriptors featurization. This will serve as a baseline as you apply more advanced featurization schemes.

It is essential for many ML methods, depending on the data type, to perform preprocessing in order to attain optimal performance. Here, we choose to normalize and truncate the inputs (X, the featurized complexes) while normalizing the output (y, the binding affinities):

To recap, the initial dataset has now been featurized, split into train and test sets, and transformed. We are now ready to learn some chemistry! To warm up, let's apply a more traditional but quite robust statistical learning technique: random forests.

Now that we've fit an Random Forest Regressor on the data, let's evaluate its performance:

Let's compare this to performance with a deep neural network.

In [ ]:
from deepchem.models import Model

task_type = "regression"
model_params = {"activation": "relu",
              "dropout": 0.,
              "momentum": .9, "nesterov": False,
              "decay": 1e-4, "batch_size": 5,
              "nb_epoch": 10}
model_name = "singletask_deep_regressor"

nb_hidden_vals = [10, 100]
learning_rate_vals = [.01, .001]
init_vals = ["glorot_uniform"]
hyperparameters = [nb_hidden_vals, learning_rate_vals, init_vals]
hyperparameter_rows = []
for hyperparameter_tuple in itertools.product(*hyperparameters):
    nb_hidden, learning_rate, init = hyperparameter_tuple
    model_params["nb_hidden"] = nb_hidden
    model_params["learning_rate"] = learning_rate
    model_params["init"] = init

    model_dir = tempfile.mkdtemp()

    r2_score = create_and_eval_model(train_dataset, test_dataset, task_type,
                                     model_params, model_name, model_dir, tasks)

    print("%s: %s" % (hyperparameter_tuple, r2_score))
    hyperparameter_rows.append(list(hyperparameter_tuple) + [r2_score])

    shutil.rmtree(model_dir)

hyperparameter_df = pd.DataFrame(hyperparameter_rows,
                               columns=('nb_hidden', 'learning_rate',
                                        'init', 'r2_score'))

hyperparameter_df